In [2]:
import graphistry
import pandas as pd
import numpy as np
import requests

TIGER = "https://halal.i.tgcloud.io"

In [3]:
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="irfnrifqi", password="Labsraw20")
#graphistry.register(key='MY_API_KEY', server='labs.graphistry.com')

In [4]:
import pyTigerGraph as tg

TG_HOST = "https://halal.i.tgcloud.io"
TG_USERNAME= "tigergraph"
TG_PASSWORD="syahnaz"
TG_GRAPH ="halal"
TG_SECRET= "gh9urnvt8aa0hk2tppic1r9vck0jmumj"

conn = tg.TigerGraphConnection(host=TG_HOST, graphname=TG_GRAPH, username=TG_USERNAME, password=TG_PASSWORD)

print(conn.getToken(TG_SECRET, "1000000")) #uses a lifetime of 1,000,000 seconds

('2sqcde9m001qku8jrf9uiriuvnjj887t', 1618998107, '2021-04-21 09:41:47')


In [5]:
nodeFood = conn.getVertexDataframe("food", limit=100)
nodeFood.head(10)

,v_id,id,food_name
0,76372,76372,polyethylene terephthalate CB 606
1,76365,76365,polyethylene terephthalate CB 600
2,77298,77298,priority
3,61083,61083,dehydrated garlic flakes
4,59534,59534,chinos coklat blueberry
5,80520,80520,sodium diacetate
6,76877,76877,potassium lactate and sodium acetate blend
7,76636,76636,pillow
8,51651,51651,ampela
9,46180,46180,Tepung Jgung Suka Suka


In [6]:
g = graphistry.edges(nodeFood).bind(source='id', destination='food_name')
g.plot()

In [7]:
queryName = "GetHalalProduct"
preInstalledResult = conn.runInstalledQuery(queryName) 
parsR = (preInstalledResult)
HalalProd = conn.vertexSetToDataFrame(parsR[0]["Product"], withId=True, withType=False)
HalalProd

,v_id,id,food_name
0,68879,68879,kue wafel isi karamel dan cokelat Chocolate St...
1,61138,61138,egg albumen powder AD
2,60659,60659,dehydrated onion
3,59766,59766,chinos coklat nanas
4,80237,80237,sodium lactate and sodium acetate blend
...,...,...,...
95,44063,44063,Soybean protein isolated XL930
96,27672,27672,MATRIX GRAPE FLAVOUR
97,45915,45915,Tong Kopi Red Velvet Drink
98,27940,27940,MATRIX MANDARIN ORANGE FLAVOUR


In [8]:
g = graphistry.edges(HalalProd).bind(source='id', destination='food_name')
g.plot()

In [9]:
query = "ProductManufactureLink"
resultTG = conn.runInstalledQuery(query)
results = resultTG[0]['@@tupleRecords']
ProdMan = pd.DataFrame(results)
ProdMan

,src,dest
0,kue wafel isi karamel dan cokelat Chocolate St...,Pt. Aaltje Boga Kulina
1,egg albumen powder AD,Taiyo Food (Tianjin) Co.
2,dehydrated onion,Laiwu Manhing Vegetables Fruits Corporation
3,chinos coklat nanas,CHINOS
4,sodium lactate and sodium acetate blend,HeNan JinDan Lactic Acid Technology Co.
...,...,...
43096,TANGO Wafer Rasa Susu bentuk Waffle,Ultra Prima Abadi - Divisi Wafer & Delicacies
43097,Marneng Jagung Original Joss,PKOPRBG JOSS
43098,Coloursea Ponceau 4R CI 16255,Centra Lautan Pewarna PT
43099,Coloursea Erythrosine CI 45430,Centra Lautan Pewarna PT


In [10]:
g = graphistry.edges(ProdMan).bind(source='src', destination='dest')
g.plot()

In [11]:
query = "ProductIngredientLink"
resultTG = conn.runInstalledQuery(query)
results = resultTG[0]['@@tupleRecords']
ProdIng = pd.DataFrame(results)
ProdIng

,src,dest
0,Zona Keripik Tempe Soy Bean Crips,Soybean
1,Zona Keripik Tempe Soy Bean Crips,Sugar
2,Zona Keripik Tempe Soy Bean Crips,Flavor Enhancer
3,Zona Keripik Tempe Soy Bean Crips,Salt
4,Zona Keripik Tempe Soy Bean Crips,Vegetable Oil
...,...,...
131335,,avena
131336,,arrowroot powder organic coconut sugar organic...
131337,,apple
131338,,Ascorbic acid


In [12]:
g = graphistry.edges(ProdIng).bind(source='src', destination='dest')
g.plot()

# Node2Vec

In [13]:
%matplotlib inline
from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from node2vec import Node2Vec
sns.set_style('whitegrid')
import pandas as pd
import warnings

In [14]:
g = nx.from_pandas_edgelist(ProdMan, source='src',target='dest')

In [15]:
mylist = ProdMan.values.tolist()
mylist

[['kue wafel isi karamel dan cokelat Chocolate Stroopwaffel ',
  'Pt. Aaltje Boga Kulina'],
 ['egg albumen powder AD', 'Taiyo Food (Tianjin) Co.'],
 ['dehydrated onion', 'Laiwu Manhing Vegetables Fruits Corporation'],
 ['chinos coklat nanas', 'CHINOS'],
 ['sodium lactate and sodium acetate blend',
  'HeNan JinDan Lactic Acid Technology  Co.'],
 ['freeze dried coffee extract', 'DeHong Hogood Coffee Co.'],
 ['bandeng presto MAMA YUS ', 'PKOPJPA JAVA FOOD'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'PT. Zoneindo Global'],
 ['Ziboxan WS80 Specification', 'Deosen Biochemical (Ordos) Ltd.'],
 ['Ziboxan IND40 Specification', 'Deosen Biochemical (Ordos) Ltd.'],
 ['Ziboxan UH Specification', 'Deosen Biochemical (Ordos) Ltd.'],
 ['Tepung Sagu Cap Hati', 'Bina Sago Lestari'],
 ['VANA BLANCA 32Q', 'Kievit Indonesia'],
 ['VANA CAPPA X760', 'Kievit Indonesia'],
 ['VANA BLANCA 60P', 'Kievit Indonesia'],
 ['Teh Wangi Cap Merak Djago', 'Kartini Teh Nasional'],
 ['Teh Wangi Gopek Etiket Coklat ', 'Gopek 

In [16]:
g.add_edges_from(mylist,weight=1,label="Manufacture")

In [ ]:
node2vec = Node2Vec(g, dimensions=20, walk_length=16,num_walks=5, workers=2)

In [ ]:
model = node2vec.fit(window=10,min_count=1)

In [ ]:
nodeFood =[x for x in ProdMan.src]
nodeFood

In [ ]:
embeddings = np.array([model.wv[x] for x in nodeFood])
embeddings

In [ ]:
tsne = TSNE(n_components=2, random_state=7, perplexity=15)
embeddings_2d = tsne.fit_transform(embeddings)

In [ ]:
figure = plt.figure(figsize=(100,100))

ax = figure.add_subplot(111)

ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

In [31]:
g2 = nx.from_pandas_edgelist(ProdIng, source='src',target='dest')

In [32]:
mylist2 = ProdIng.values.tolist()
mylist2

[['Zona Keripik Tempe Soy Bean Crips ', 'Soybean'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Sugar'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Flavor Enhancer'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Salt'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Vegetable Oil'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Water'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Chili'],
 ['Zona Keripik Tempe Soy Bean Crips ', 'Yeast'],
 ['Noodle Bowl', 'Red Pepper Oil'],
 ['Noodle Bowl', 'Palm Oil'],
 ['Noodle Bowl', 'Potassium carbonate'],
 ['Noodle Bowl', 'Kelp Extract Powder'],
 ['Noodle Bowl', 'Glucose'],
 ['Noodle Bowl', 'Soybean'],
 ['Noodle Bowl', 'Green Onion'],
 ['Noodle Bowl', 'Beta carotene'],
 ['Noodle Bowl', 'Disodium Inosinate'],
 ['Noodle Bowl', 'Calcium'],
 ['Noodle Bowl', 'Sodium Poliphospate'],
 ['Noodle Bowl', 'Modified starch'],
 ['Noodle Bowl', 'Sugar'],
 ['Noodle Bowl', 'Wheat Gluten'],
 ['Noodle Bowl', 'Dextrin'],
 ['Noodle Bowl', 'Beef Bone Extract'],
 ['Noodle Bowl', 'Roasted Layer

In [33]:
g2.add_edges_from(mylist2,weight=1,label="Ingredient")

In [ ]:
node2vec = Node2Vec(g2, dimensions=20, walk_length=16,num_walks=100, workers=2)

In [ ]:
import streamlit as st
import altair as alt
from vega_datasets import data
import time

source = data.cars()
chart = st.empty()


for i in source.index:
    data_to_be_added = source.iloc[0: i + 1, :]

    x = alt.Chart(data_to_be_added).mark_circle(size=i * 10).encode(
        x='Horsepower',
        y='Miles_per_Gallon',
        color='Origin',
        tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
    ).interactive()

    time.sleep(0.2)

    chart.altair_chart(x)